<a href="https://colab.research.google.com/github/reckoning-machines/sec_text_nlp/blob/master/r_getter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# why use R here?  
# edgarWebR pulls sections really well
# do i want to find a python library for the same thing?  sure.
# do i want to write a python utils file for the same thing?  not really but we may have to!

#first pull the ticker list with google drive handler lib
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo
from colab_util import *
drive_handler = GoogleDriveHandler()

drive_handler.download('test_ticker_list.csv', target_path='test_ticker_list.csv')

import pandas as pd
df_tickers = pd.read_csv('test_ticker_list.csv')
#df_tickers = df_tickers[df_tickers['Symbol']=='CVX']
#df_tickers.to_csv('test_ticker_list.csv')


Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [2]:
# activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [3]:
%%R

#R GETTER.  For this example, only one ticker (head,1) is pulled 
#file is saved into local 
#then python code cell pushes file to google sheet
#devtools::install_github("tidyverse/googlesheets4")

#devtools::install_version("xml2", version = "1.2.2", repos = "http://cran.us.r-project.org")
#file creates a set of csv from ticker list which include metadata & text data.


  #devtools::install_packages('trinker/textclean')
  devtools::install_github("mwaldstein/edgarWebR")
  devtools::install_github("r-lib/xml2") #this for edgarWebR 
  devtools::install_github('trinker/textclean')

  library(edgarWebR) #this is an up to date library with an active maintainer.
  library(xml2)
  library(knitr)
  library(dplyr)
  library(purrr)
  library(rvest)
  library(tidyr)
  library(readr)
  #library(textshape)
  #library(lexicon)
  library(textclean)

  #library(log4r) TODO logging file.
  #library(googlesheets4)
  #gs4_deauth()
  #tickers list
  #str_sheet <- "1_xcDVKjR2jqE-w5LqxIWnrDYstpSrE5nFSSY0WXNOVE"
  #df_tickers <- read_sheet(str_sheet)

  #helper functions

  get_filings_links <-function(str_ticker) {
    df_filings <- company_filings(str_ticker, type = "10-", count = 20)
    df_filings <- df_filings[df_filings$type == "10-K" | df_filings$type == "10-Q", ]
    df_filing_infos <- map_df(df_filings$href, filing_information)
    df_filings <- bind_cols(df_filings, df_filing_infos)
    return(head(as_tibble(df_filings),6))
  }

  get_section_text <- function(str_href, str_section, str_search) {
    
    df_filing_documents <- filing_documents(str_href)
    str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
    doc <- parse_filing(str_doc_href)

    df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
    #we could do some text preprocessing here.

    df_txt <- as_tibble(df_txt) %>%
      mutate(text = textclean::strip(text)) %>%
      mutate(section = str_search)

    return(df_txt)
  }



R[write to console]: Downloading GitHub repo mwaldstein/edgarWebR@master



curl (4.2 -> 4.3) [CRAN]


R[write to console]: Installing 1 packages: curl

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/curl_4.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 673779 bytes (657 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write

✔  checking for file ‘/tmp/Rtmpl4WZgl/remotes7e37c14ba9/mwaldstein-edgarWebR-e7fa70e/DESCRIPTION’
─  preparing ‘edgarWebR’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘edgarWebR_1.0.2.tar.gz’
   


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: Downloading GitHub repo r-lib/xml2@master



...

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: Downloading GitHub repo trinker/textclean@master



data.table (NA    -> 1.12.8) [CRAN]
english    (NA    -> 1.2-5 ) [CRAN]
glue       (1.4.0 -> 1.4.1 ) [CRAN]
lexicon    (NA    -> 1.2.1 ) [CRAN]
mgsub      (NA    -> 1.7.1 ) [CRAN]
qdapRegex  (NA    -> 0.7.2 ) [CRAN]
textshape  (NA    -> 1.7.1 ) [CRAN]
NLP        (NA    -> 0.2-0 ) [CRAN]
zoo        (NA    -> 1.8-8 ) [CRAN]
dtt        (NA    -> 0.1-2 ) [CRAN]
tidyr      (1.0.2 -> 1.0.3 ) [CRAN]
tidyselect (1.0.0 -> 1.1.0 ) [CRAN]
slam       (NA    -> 0.1-47) [CRAN]
vctrs      (0.2.4 -> 0.3.0 ) [CRAN]
pillar     (1.4.3 -> 1.4.4 ) [CRAN]
syuzhet    (NA    -> 1.0.4 ) [CRAN]


R[write to console]: Installing 16 packages: data.table, english, glue, lexicon, mgsub, qdapRegex, textshape, NLP, zoo, dtt, tidyr, tidyselect, slam, vctrs, pillar, syuzhet

R[write to console]: Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/data.table_1.12.8.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 4948391 bytes (4.7 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to

✔  checking for file ‘/tmp/Rtmpl4WZgl/remotes7e46add6f6/trinker-textclean-184d786/DESCRIPTION’
─  preparing ‘textclean’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  looking to see if a ‘data/datalist’ file should be added
─  building ‘textclean_0.9.5.tar.gz’
   


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘rvest’


R[write to console]: The following object is masked from ‘package:purrr’:

    pluck


R[write to console]: 
Attaching package: ‘readr’


R[write to console]: The following object is masked from ‘package:rvest’:

    guess_encoding




In [4]:
%%R

get_mdna_text <- function(str_href) {
  print(str_href)
  
  #default search
  str_section = 'item 2|item 7'
  str_search = 'discussion'  
  
  df_filing_documents <- filing_documents(str_href)
  str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
  doc <- parse_filing(str_doc_href)

  #return(doc)
    
  df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
  #if default search fails, use a dictionary attempt
  if (nrow(df_txt) == 0) {
    print('going to backup')
    # paired vector of start and ending text to slice if found

    #JPM and #ECL
    vec_start_end <- c('EXECUTIVE OVERVIEW'='CONSOLIDATED RESULTS OF OPERATIONS',
                       'The following management discussion and analysis'='NON-GAAP FINANCIAL MEASURES')
    
    #this would be case sensitive
    for (start_text in names(vec_start_end)) {
      
      end_text = as.character(vec_start_end[start_text])
      
      print(paste0('trying from ',start_text))
      print(paste0('to ',end_text))
      
      i_start = as.integer(which(grepl(start_text, doc$text))) #this should be a loop for each dictionary item
      i_end = as.integer(which(grepl(end_text, doc$text)))
            
      if (length(i_start) != 0 & length(i_end) != 0) {
        df_txt = doc[i_start:i_end,]
      }
      
    }
    
  }
  #we could do some text preprocessing here.
}

get_document_text <- function(str_ticker, force = FALSE) { #not using force yet
  start_time <- Sys.time()
  
  print(str_ticker)
  
  str_write_name <- paste0('sec_data_folder/',str_ticker)
  #paste0("~/data/",str_ticker,"_sec_text")

  print("get filings links ...")

  df_filings <- get_filings_links(str_ticker)

  #print(df_filings)

  print("get section text ...")

  df_data <- (df_filings) %>% 
    rowwise() %>%
    mutate(nest_discussion = map(.x = href, .f = get_mdna_text)) %>%
    ungroup() %>%
    group_by(period_date) %>%
    arrange(desc(period_date))
  
  #jenky - find a rowwise application
  a <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_discussion) %>%
    unnest(nest_discussion)

  print("write to local csv  ...")

  #ss <- gs4_create(str_write_name)
  
  #df_data <- rbind(a,b,c,d) %>%
  #  googlesheets4::sheet_write(ss, sheet = "sec_data")
  
  df_data <- a %>%
    write_csv(paste0(str_write_name,".csv"))

  end_time <- Sys.time()

  print(end_time - start_time)

  return(df_data)
}

#long run.
df_tickers <- read_csv('test_ticker_list.csv')
dir.create('sec_data_folder', showWarnings = FALSE)

#file creates a set of csv from ticker list which include metadata & text data.
df_tickers <- head(df_tickers,1)

df_data <- map_df(df_tickers$Symbol, get_document_text)


R[write to console]: Parsed with column specification:
cols(
  X1 = col_double(),
  `Unnamed: 0` = col_double(),
  `GICS Sector` = col_character(),
  Symbol = col_character(),
  mkt_cap = col_double()
)



[1] "CVX"
[1] "get filings links ..."
[1] "get section text ..."
[1] "https://www.sec.gov/Archives/edgar/data/93410/000009341020000020/0000093410-20-000020-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/93410/000009341020000010/0000093410-20-000010-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/93410/000009341019000038/0000093410-19-000038-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/93410/000009341019000029/0000093410-19-000029-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/93410/000009341019000014/0000093410-19-000014-index.htm"
[1] "https://www.sec.gov/Archives/edgar/data/93410/000009341019000008/0000093410-19-000008-index.htm"
[1] "write to local csv  ..."
Time difference of 20.14163 secs


In [5]:
#push files to google drive
import os.path
from os import path
from colab_util import *
drive_handler = GoogleDriveHandler()

sec_folder_id = drive_handler.create_folder('sec_data_folder')

import pandas as pd
df_tickers = pd.read_csv('test_ticker_list.csv')

for i, row in df_tickers.iterrows():
  str_ticker = row['Symbol']
  
  print('working for:'+str_ticker+".csv ...")

  str_to_file = 'sec_data_folder'
  str_from_file = 'sec_data_folder/'+str_ticker+'.csv'

  if path.exists(str_from_file): # a rare example of error handling
    drive_handler.upload(str_from_file, parent_path=str_to_file,overwrite = False)

print("done!")


sec_data_folder already exists
working for:CVX.csv ...
done!
